In [1]:
import sys, os, time
from pathlib import Path
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Paths importantes
PDF_PATH = PROJECT_ROOT / "data" / "pdfs" / "4349.pdf"
EMBEDDINGS_PATH = PROJECT_ROOT / "data" / "embeddings"


In [3]:
from src.utils.vector_store import VectorStore

In [4]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [5]:
# Cargar el PDF
loader = PyPDFLoader(str(PDF_PATH))
documents = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Dividir los documentos
chunks = text_splitter.split_documents(documents)

In [7]:
vector_store = VectorStore(collection_name="Atlas_eolico_Colombia-Ollama", embedding_function=embeddings)

In [8]:
ids = vector_store.add_documents(chunks)

In [9]:
# Ejemplo de búsqueda básica
query = "¿Cuál es el tema principal del documento?"

results = vector_store.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f"\n--- Resultado {i} ---")
    print(f"📄 Página: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 Contenido:\n{doc.page_content[:400]}...")
    print()


--- Resultado 1 ---
📄 Página: 1
📝 Contenido:
Olga Cecilia González Gómez, Ing. Geógrafo
 Esperanza Pardo Pardo, Ing. Agrónomo, SIG  
CONSULTORES
 E
SPECIALISTA
 Jorge Aníbal Zea Mazo, Meteorólogo
TÉCNICOS DIGITADORES
 Angélica Galeano Díaz
 José Álvaro Fajardo Arévalo
 Avelino Suárez Rubio
 Germán Andrés Franco Aguirre
 Jorge Alonso Semma
 Lázaro Supelano Castelblanco
 Andrés Riaño Flechas
 Moisés Valencia Valencia
 Zulma Inés Albarracín Mor...


--- Resultado 2 ---
📄 Página: 147
📝 Contenido:
resultante al tope de la Capa Límite Planetaria.
4.1  METODOLOGÍA PARA EL ATLAS 
DE VIENTO
El desarrollo del trabajo requirió un amplio 
conocimiento de la climatología del país, en 
especial conocimiento de la posición media de 
los sistemas sinópticos que dominan el estado 
del tiempo en cada uno de los meses del año, 
particularmente la posición media de la Zona 
de Convergencia Intertropical (...


--- Resultado 3 ---
📄 Página: 11
📝 Contenido:
formación de otros países fronterizos como 
Venez

In [10]:
# Búsqueda con scores de similitud
query = "clima"

results_with_scores = vector_store.similarity_search_with_score(query, k=5)

for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"\n--- Resultado {i} (Score: {score:.4f}) ---")
    print(f"📄 Página: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 Contenido:\n{doc.page_content[:300]}...")


--- Resultado 1 (Score: 0.6854) ---
📄 Página: 130
📝 Contenido:
109 ALBÁN VALLE EL CAIRO 04°46’N 76°13’W
110 APTO. PUERTO CARREÑO VICHADA PUERTO CARREÑO 06°11’N 67°29’W
111 LAS GAVIOTAS VICHADA SAN JOSÉ DE OCUNE 04°33’N 70°56’W
BIBLIOGRAFÍA 
1. W . J. Gibbs. Deﬁ niendo el clima. Revista oﬁ  cial de la Organización Meteorológica Mundial ,
OMM, volumen 36, No. 4, ...

--- Resultado 2 (Score: 0.7760) ---
📄 Página: 111
📝 Contenido:
en sectores con topografías que contrastan entre 
formaciones de valles y montañas.
1.3.4.1  BRISAS DE MAR 
Resultan del calentamiento desigual de las 
superﬁ cies de tierra y agua. En un día soleado, 
puede observarse un cambio notable en la 
temperatura temprano cuando se inicia el 
calentamiento ...

--- Resultado 3 (Score: 0.8007) ---
📄 Página: 129
📝 Contenido:
89 VILLA LEIVA SANTANDER SABANA DE TORRES 07°27’N 73°33’W
90 UNIV. IND. SANTANDER SANTANDER BUCARAMANGA 07°08’N 73°06’W
91 APTO. PALONEGRO SANTANDER LEBRIJA 07°08’N 73°11’W
92 EL CUCHARO SANTANDER PIN

In [11]:
llm = ChatOllama(model="llama3.2:1b", temperature=0,
    num_predict=256,  # Limitar tokens de salida
    num_ctx=2048,     # Reducir ventana de contexto
    top_k=10,         # Reducir para más velocidad
    top_p=0.9
)

In [12]:
# Crear el retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

# Template para RAG
template = """Responde la pregunta basándote únicamente en el siguiente contexto:

{context}

Pregunta: {question}

Respuesta:"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Crear la cadena RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Cadena RAG configurada correctamente")

✅ Cadena RAG configurada correctamente


In [13]:
# Hacer una pregunta usando RAG con medición de tiempos
question = "¿Cuál es el tema principal del documento?"

print(f"❓ Pregunta: {question}")
print("\n" + "=" * 60)

# Medir tiempo de recuperación
start_retrieval = time.time()
docs = retriever.invoke(question)
retrieval_time = time.time() - start_retrieval

# Formatear documentos
context = format_docs(docs)

print(f"\n⏱️  Tiempo de recuperación: {retrieval_time:.3f}s")
print(f"📚 Documentos recuperados: {len(docs)}")

# Medir tiempo de generación del LLM
print("\n💬 Respuesta:")
start_llm = time.time()
response = llm.invoke(prompt.format(context=context, question=question))
llm_time = time.time() - start_llm

print(response.content)

# Resumen de tiempos
total_time = retrieval_time + llm_time
print("\n" + "=" * 60)
print("📊 Métricas de rendimiento:")
print(f"   🔍 Recuperación:  {retrieval_time:.3f}s ({retrieval_time/total_time*100:.1f}%)")
print(f"   🤖 Generación:    {llm_time:.3f}s ({llm_time/total_time*100:.1f}%)")
print(f"   ⏱️  Total:         {total_time:.3f}s")

❓ Pregunta: ¿Cuál es el tema principal del documento?


⏱️  Tiempo de recuperación: 0.054s
📚 Documentos recuperados: 4

💬 Respuesta:
El tema principal del documento es la creación y análisis de un modelo para predecir los patrones de viento en una región específica utilizando datos meteorológicos.

📊 Métricas de rendimiento:
   🔍 Recuperación:  0.054s (0.2%)
   🤖 Generación:    25.111s (99.8%)
   ⏱️  Total:         25.165s
